In [ ]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openpyxl
import pandas as pd


df = pd.read_excel('/content/drive/MyDrive/train.xlsx') # Use read_excel to read .xlsx files
print(df.head())


                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization, Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential

# Load Excel file
df = pd.read_excel("/content/drive/MyDrive/train.xlsx", sheet_name=0)

# Define features (X) and labels (y)
X = df['comment_text'].astype(str) # Convert the column to string type to ensure no float values are present
y = df[df.columns[2:]].values  # Assuming the columns from index 2 onwards are labels

# Text Vectorization setup
MAX_FEATURES = 20000  # Reduced vocabulary size for smaller dataset
SEQUENCE_LENGTH = 1800  # Sequence length of the text input

vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=SEQUENCE_LENGTH,
                               output_mode='int')

vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)

# Dataset creation and transformation
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(10000)  # Shuffle size equals dataset size
dataset = dataset.batch(32)  # Increase batch size for small dataset
dataset = dataset.prefetch(8)  # Prefetch to prevent bottlenecks

# Train/Validation/Test split
train_size = int(0.7 * len(dataset))  # 70% training
val_size = int(0.2 * len(dataset))  # 20% validation
test_size = len(dataset) - train_size - val_size  # 10% test

train = dataset.take(train_size)
val = dataset.skip(train_size).take(val_size)
test = dataset.skip(train_size + val_size)

# Model architecture
model = Sequential()

# Add Input layer with the correct shape
model.add(tf.keras.Input(shape=(SEQUENCE_LENGTH,)))  # Explicitly define input shape

# Embedding Layer
model.add(Embedding(MAX_FEATURES + 1, 32))  # Embedding layer for input text

# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))  # LSTM to process text sequence

# Fully Connected Layers (Feature extraction layers)
model.add(Dense(128, activation='relu'))  # 128 units, ReLU activation
model.add(Dense(256, activation='relu'))  # 256 units, ReLU activation
model.add(Dense(128, activation='relu'))  # 128 units, ReLU activation

# Output Layer
model.add(Dense(6, activation='sigmoid'))  # Output for multi-label classification

# Compile Model
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

# Model Summary
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)             │ (None, 1800, 32)            │         640,032 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_11 (Bidirectional)     │ (None, 64)                  │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_47 (Dense)                     │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 731,686 (2.79 MB)

 Trainable params: 731,686 (2.79 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train, epochs=10, validation_data=val)


Epoch 1/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 460s 2s/step - loss: 0.2467 - val_loss: 0.0991
Epoch 2/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 454s 2s/step - loss: 0.0857 - val_loss: 0.0752
Epoch 3/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 449s 2s/step - loss: 0.0622 - val_loss: 0.0603
Epoch 4/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 450s 2s/step - loss: 0.0587 - val_loss: 0.0527
Epoch 5/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 504s 2s/step - loss: 0.0521 - val_loss: 0.0481
Epoch 6/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 452s 2s/step - loss: 0.0506 - val_loss: 0.0472
Epoch 7/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 451s 2s/step - loss: 0.0431 - val_loss: 0.0420
Epoch 8/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 500s 2s/step - loss: 0.0396 - val_loss: 0.0376
Epoch 9/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 498s 2s/step - loss: 0.0386 - val_loss: 0.0370
Epoch 10/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 501s 2s/step - loss: 0.0385 - val_loss: 0.0402


In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()
for batch in test.as_numpy_iterator():
    # Unpack the batch
    X_true, y_true = batch
    # Make a prediction
    yhat = model.predict(X_true)

    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 903ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 406ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')


Precision: 0.9035087823867798, Recall:0.6866666674613953, Accuracy:0.27272728085517883
